In [31]:
using Statistics
using LinearAlgebra
using Random
using Plots
using QuadGK
using LaTeXStrings

In [32]:
function def_nonDiag(d, mtr::Matrix{Float64}, nondiag::Float64)
    for i in 1:d, j in 1:d
        if i != j
            mtr[i, j] = nondiag
        end
    end
    return mtr
end

def_nonDiag (generic function with 1 method)

$$
v(\vec{S}(T)) = \max \Bigl\lbrace \min \lbrace S^1_T, \ldots, S^d_T \rbrace - K, 0 \Bigl\rbrace
$$

In [33]:
function payoff_v(x::Vector{Float64}, K::Float64)
    return maximum([(minimum(exp.(x)) - K), 0.0])
end
;

$$
q(\vec{x}|\vec{x}_0) = \frac{1}{\sqrt{(2 \pi)^d \det \Sigma}} \exp\left(-\frac{1}{2}\left(\vec{x}-\vec{\mu}\right)^T \Sigma^{-1} \left(\vec{x}-\vec{\mu}\right)\right)
$$

$$
\mu_j = x_0^j+r_j T-\frac{1}{2} \sigma_j^2 \Sigma_{j j} T 
$$

In [34]:
function normal_dstrb_q(d::Int, x::Vector{Float64}, x0::Vector{Float64}, σ::Vector{Float64}, Σ::Matrix{Float64}, T::Float64, r::Float64)
    #μ = [x0[i] + r * T - σ[i]^2 * Σ[i,i] * T / 2 for i in 1:d]
    μ = [x0[i] + r * T - Σ[i,i] * T / 2 for i in 1:d]
    mtrx = transpose(x .- μ) * inv(Σ) * (x .- μ)
    
    return exp(-mtrx/2)/sqrt((2*π)^d * det(Σ))
end

normal_dstrb_q (generic function with 1 method)

In [35]:
function func(idxs::Vector{Int}, f::Function, N::Int, cut::Float64, slice::Float64)
    return f( cut .* ( idxs ) .+ slice )
end

func (generic function with 1 method)

In [36]:
d = 2
T = 1.0
r = 0.01
K = 100.0
s0 = fill(100.0, d)
x0 = log.(s0)
σ = fill(0.2, d)

nondiag = 1/3
Σ_ = def_nonDiag(d, Matrix{Float64}(I, d, d), nondiag)
σ_ = Matrix{Float64}(I, d, d) .* σ
Σ = σ_ * Σ_ * σ_

2×2 Matrix{Float64}:
 0.04       0.0133333
 0.0133333  0.04

In [37]:
v(x) = payoff_v(x, K)
q(x) = normal_dstrb_q(d, x, x0, σ, Σ, T, r)

q (generic function with 1 method)

In [38]:
# 被積分関数の定義 d=1
function f(x)
    return v([x]) * q([x])  # Example: x*y * exp(-x^2 - y^2)
end

# 積分区間の指定
a = log(K) - 10 * σ * sqrt(T)   # 下限
b = log(K) + 10 * σ * sqrt(T)    # 上限

# 数値積分の実行
result, error = quadgk(f, a, b)

println("積分結果: ", result)
println("推定誤差: ", error)

MethodError: MethodError: no method matching -(::Float64, ::Vector{Float64})
For element-wise subtraction, use broadcasting with dot syntax: scalar .- array

Closest candidates are:
  -(::Real, !Matched::Complex{Bool})
   @ Base complex.jl:321
  -(::Real, !Matched::Complex)
   @ Base complex.jl:333
  -(!Matched::SparseArrays.AbstractSparseMatrixCSC, ::Array)
   @ SparseArrays /Applications/Julia-1.10.app/Contents/Resources/julia/share/julia/stdlib/v1.10/SparseArrays/src/sparsematrix.jl:2247
  ...


In [39]:
res = result * exp(- r * T)

UndefVarError: UndefVarError: `result` not defined

In [40]:
86.46619501281435 - 8.433318690109608

78.03287632270474

In [41]:
# Define a 2D function to integrate
function f(x, y)
    return v([x, y]) * q([x, y])  # Example: x*y * exp(-x^2 - y^2)
end

# Set the integration limits
x_min, x_max = 0.0, 10.0
y_min, y_max = 0.0, 10.0

# Perform 2D integration using nested QuadGK
result, error = quadgk(x -> quadgk(y -> f(x, y), y_min, y_max)[1], x_min, x_max)

# Output the result
println("Integral result: ", result)
println("Estimated error: ", error)

Integral result: 3.3770745186968596
Estimated error: 4.603621238347125e-8


In [42]:
# Define a 3D function to integrate
function f(x, y, z)
    return v([x, y, z]) * q([x, y, z])  # Example: x*y*z * exp(-x^2 - y^2 - z^2)
end

# Set the integration limits
x_min, x_max = 0.0, 10.0
y_min, y_max = 0.0, 10.0
z_min, z_max = 0.0, 10.0

# Perform 3D integration using nested QuadGK
result, error = quadgk(x -> quadgk(y -> quadgk(z -> f(x, y, z), z_min, z_max)[1], y_min, y_max)[1], x_min, x_max)

# Output the result
println("Integral result: ", result)
println("Estimated error: ", error)

InterruptException: InterruptException:

In [43]:
res = result * exp(- r * T)

3.3434720657943777

In [44]:
28.340948795376008

28.340948795376008

In [45]:
res - 28.340948795376008

-24.99747672958163

In [46]:
localdims = fill(N, d)
firstpivot = rand(1:N, d)

v_tci = TCI.ThreadedBatchEvaluator{Float64}(v_pre, localdims)
q_tci = TCI.ThreadedBatchEvaluator{Float64}(q_pre, localdims)

UndefVarError: UndefVarError: `N` not defined

In [47]:
tol = 1e-6

1.0e-6

In [48]:
tci_time = @elapsed begin
    @time qtt_v, errors_v = tci_oneshot(v_tci, d, localdims, firstpivot, tol)
    @time qtt_q, errors_q = tci_oneshot(q_tci, d, localdims, firstpivot, tol)
end

UndefVarError: UndefVarError: `tci_oneshot` not defined

In [49]:
M1 = qtt_v.sitetensors
M2 = qtt_q.sitetensors

UndefVarError: UndefVarError: `qtt_v` not defined

In [50]:
smpl_indices_v = []
smpl_indices_q = []
for i in 1:100
    x = Int(N/100) * i
    push!(smpl_indices_v, [Int(N*0.8), x])
    push!(smpl_indices_q, [Int(N*0.6), x])
end
@show smpl_indices_v
@show smpl_indices_q
;

UndefVarError: UndefVarError: `N` not defined

In [51]:
plot_v = plot(
    collect(Int(N/100):Int(N/100):N),
    real.(v_pre.(smpl_indices_v))
)

plot!(
    plot_v,
    collect(Int(N/100):Int(N/100):N),
    real.(evaluate_options_(smpl_indices_v, M1)[1])
)

UndefVarError: UndefVarError: `N` not defined

In [52]:
plot(
    collect(Int(N/100):Int(N/100):N),
    real.(v_pre.(smpl_indices_v)) .- real.(evaluate_options_(smpl_indices_v, M1)[1])
)

UndefVarError: UndefVarError: `N` not defined

In [53]:
plot_q = plot(
    #yscale = :log10,
    collect(20:20:2000),
    real.(q_pre.(smpl_indices_q))
)

plot!(
    plot_q,
    collect(20:20:2000),
    real.(evaluate_options_(smpl_indices_q, M2)[1])
)

UndefVarError: UndefVarError: `q_pre` not defined

In [54]:
plot(
    collect(20:20:2000),
    real.(q_pre.(smpl_indices_q)) .-  real.(evaluate_options_(smpl_indices_q, M2)[1])
)

UndefVarError: UndefVarError: `q_pre` not defined

In [55]:
num_samples = 100
random_combinations = qo.generate_random_combinations(N, d, num_samples)
;

UndefVarError: UndefVarError: `qo` not defined

In [56]:
@show maximum(real.(v_pre.(random_combinations)) .- real.(evaluate_options_(random_combinations, M1)[1]))
@show maximum(real.(q_pre.(random_combinations)) .- real.(evaluate_options_(random_combinations, M2)[1]))

UndefVarError: UndefVarError: `v_pre` not defined

$$
V(\vec{p})=\mathbb{E}\left[e^{-rT}v(\vec{S}(T))\middle|\vec{S}_0\right]=e^{-rT} \int_{-\infty}^{\infty} v(\exp(\vec{x}))q(\vec{x}|\vec{x}_0)dx
$$

In [57]:
function create_AB(A::Array{ComplexF64,3}, B::Array{ComplexF64,3})
    # A_{i j k}
    # B_{l j m}
    dim_i, dim_j_A, dim_k = size(A)
    dim_l, dim_j_B, dim_m = size(B)
    @assert dim_j_A == dim_j_B "物理次元（j）が一致しません"

    # 縮約後のテンソルAB_{i l k m}を作成
    AB = zeros(ComplexF64, dim_i, dim_l, dim_k, dim_m)
    for j in 1:dim_j_A
        for i in 1:dim_i
            for l in 1:dim_l
                for k in 1:dim_k
                    for m in 1:dim_m
                        AB[i, l, k, m] += A[i, j, k] * B[l, j, m]
                    end
                end
            end
        end
    end

    # インデックスの結合： (i, l) -> x1, (k, m) -> x2
    #dim_x1 = dim_i * dim_l
    #dim_x2 = dim_k * dim_m
    #AB_matrix = reshape(AB, dim_x1, dim_x2)

    #return AB_matrix
    return AB
end


create_AB (generic function with 1 method)

In [58]:
function mps_inner_product_custom(M1::Vector{Array{ComplexF64,3}}, M2::Vector{Array{ComplexF64,3}})
    N = length(M1)
    @assert length(M2) == N "MPSの長さが一致しません"

    # 初期化
    M = nothing  # 最初は未定義
    R = 0.0

    for i in 1:N
        A = M1[i]  # A_{ijk}
        B = M2[i]  # B_{ljm}
        # AB_{x1, x2} を作成
        AB = create_AB(A, B)

        dim_k, dim_m, dim_o, dim_p = size(AB)
        @show size(AB)

        #if M === nothing
        #    # 最初の行列
        #    M = AB
        #else
        #    # 行列積を計算
        #    M = M * AB
        #end
        dim_i = 1
        dim_l = 1

        if M === nothing
            # 最初の行列
            M = AB
            @show size(M)
        else
            @show dim_i, dim_l, dim_k, dim_m, dim_o, dim_p
            for i in 1:dim_i
                for l in 1:dim_l
                    for k in 1:dim_k
                        for m in 1:dim_m
                            for o in 1:dim_o
                                for p in 1:dim_p
                                    R += M[i, l, k, m] * AB[k, m, o, p]
                                end
                            end
                        end
                    end
                end
            end
        end
    end

    @show R

    # 最終的な結果はスカラー値になるはず
    #@show size(M)
    #@assert size(M) == (1, 1, 1, 1) "最終的な結果がスカラーではありません"
    return R
end


mps_inner_product_custom (generic function with 1 method)

In [59]:
inner_product = mps_inner_product_custom(M1, M2)
inner_product * exp(- r * T) * (cut^d)

UndefVarError: UndefVarError: `M1` not defined